In [64]:
# costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
# [9,0,21,22,36,52,24,51,36,37,41,30,20],    
# [14,21,0,25,38,5,31,7,36,43,29,7,6],    
# [23,22,25,0,42,12,35,17,44,31,31,11,6],
# [32,36,38,42,0,22,37,16,46,37,29,13,14],   
# [50,52,5,12,22,0,41,23,10,39,9,17,16],   
# [21,24,31,35,37,41,0,26,21,19,10,25,12],  
# [49,51,7,17,16,23,26,0,30,28,16,27,12],   
# [30,36,36,44,46,10,21,30,0,25,22,10,20],    
# [27,37,43,31,37,39,19,28,25,0,20,16,8],   
# [35,41,29,31,29,9,10,16,22,20,0,10,10],   
# [28,30,7,11,13,17,25,27,10,16,10,0,10],
# [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

# len(costMatrix)

def read_vrpspd_file(file_path):
    with open(file_path, "r") as f:
        lines = [line.strip() for line in f.readlines() if line.strip() and "~" not in line]

    cost_matrix = []
    delivery = []
    pickup = []
    vehicle_capacities = []

    i = 0

    # 1. Read cost matrix
    if "Cost matrix" in lines[i]:
        i += 1

    # đọc từng dòng số -> ma trận
    while i < len(lines) and "Delivery quantities" not in lines[i]:
        row = list(map(float, lines[i].split()))
        cost_matrix.append(row)
        i += 1

    # 2. Read delivery
    if "Delivery quantities" in lines[i]:
        i += 1

    delivery = list(map(int, map(float, lines[i].split())))
    i += 1

    # 3. Read pickup
    if "Pick-up quantities" in lines[i]:
        i += 1

    pickup = list(map(int, map(float, lines[i].split())))
    i += 1

    # 4. Read vehicle capacity (LIST of capacities)
    if "Vehicle capacity" in lines[i]:
        i += 1
        vehicle_capacities = list(map(float, lines[i].split()))
        i += 1

    # === ADD DEPOT ===
    delivery = [0] + delivery
    pickup = [0] + pickup

    # === FINAL VARIABLES ===
    numberOfVehicles = len(vehicle_capacities)

    # if all vehicles have same capacity -> use that as capacityOfVehicle
    if len(set(vehicle_capacities)) == 1:
        capacityOfVehicle = vehicle_capacities[0]
    else:
        # different capacity vehicles
        capacityOfVehicle = max(vehicle_capacities)

    return cost_matrix, delivery, pickup, vehicle_capacities, capacityOfVehicle, numberOfVehicles


# ===== USAGE =====
file_path = "D:\\VRP\\VRPSPD\\class7\\CMT1x.txt"   # <- thay đường dẫn file thật của bạn
costMatrix, demand, pickup, vehicle_caps, capacityOfVehicle, numberOfVehicles = read_vrpspd_file(file_path)

print("\n✅ costMatrix =", costMatrix)
print("\n✅ demand =", demand)
print("\n✅ pickup =", pickup)
print("\n✅ vehicle_capacities =", vehicle_caps)
print("✅ capacityOfVehicle =", capacityOfVehicle)
print("✅ numberOfVehicles =", numberOfVehicles)
print(len(pickup))



✅ costMatrix = [[0.0, 13.89, 21.02, 32.56, 17.2, 14.14, 11.4, 26.42, 22.02, 23.09, 28.32, 12.04, 8.06, 29.15, 18.11, 24.74, 22.02, 17.26, 14.76, 31.9, 32.45, 32.06, 20.81, 22.02, 25.06, 23.09, 28.16, 8.0, 29.97, 29.12, 30.87, 29.83, 10.0, 34.0, 31.78, 39.41, 43.93, 18.11, 15.81, 38.29, 42.2, 30.48, 31.32, 34.66, 25.0, 31.32, 2.24, 9.43, 15.81, 21.63, 26.17], [15.279, 0.0, 12.37, 22.7443, 32.479, 22.6864, 18.6514, 26.5251, 13.7588, 25.6664, 34.7464, 12.9664, 22.2164, 43.3895, 28.5614, 36.6364, 21.1662, 30.9868, 28.9695, 47.179, 22.3416, 29.0844, 10.5343, 24.4351, 30.74, 33.11, 18.87, 9.5914, 20.5232, 23.5138, 34.1564, 22.1, 6.5664, 43.984, 33.0767, 29.4823, 31.9918, 33.389, 21.421, 45.8916, 57.479, 45.759, 46.599, 37.1451, 40.279, 43.5477, 17.519, 24.709, 15.2151, 28.8616, 26.6767], [24.07, 12.37, 0.0, 21.267, 37.5376, 24.2508, 29.0, 38.8951, 26.1288, 16.7962, 28.8162, 15.0946, 25.0076, 51.09, 38.28, 38.2008, 8.7962, 34.06, 36.67, 53.37, 13.7256, 18.3538, 14.3505, 36.57, 42.56, 43.99, 

In [65]:
# Clarke-Wright Savings Heuristic for Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSPD)
# Savings-based VRPSPD (Gajpal & Abad style)
from scipy.spatial import distance
import numpy as np

# xCoordinates = [162, 218, 218, 201, 214, 224, 210, 104, 126, 119, 129, 126, 125, 116, 126, 125, 119, 115, 153, 175, 180, 159, 188, 152, 215, 212, 188, 207, 184, 207]
# yCoordinates = [354, 382, 358, 370, 371, 370, 382, 354, 338, 340, 349, 347, 346, 355, 335, 355, 357, 341, 351, 363, 360, 331, 357, 349, 389, 394, 393, 406, 410, 392]

# costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
# for i in range(len(xCoordinates)):
#     for j in range(len(yCoordinates)):
#         costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

# demand = [0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
# pickup = [1000, 0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150]
# capacityOfVehicle = 4500
# numberOfVehicles = 4

# costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
# [9,0,21,22,36,52,24,51,36,37,41,30,20],    
# [14,21,0,25,38,5,31,7,36,43,29,7,6],    
# [23,22,25,0,42,12,35,17,44,31,31,11,6],
# [32,36,38,42,0,22,37,16,46,37,29,13,14],   
# [50,52,5,12,22,0,41,23,10,39,9,17,16],   
# [21,24,31,35,37,41,0,26,21,19,10,25,12],  
# [49,51,7,17,16,23,26,0,30,28,16,27,12],   
# [30,36,36,44,46,10,21,30,0,25,22,10,20],    
# [27,37,43,31,37,39,19,28,25,0,20,16,8],   
# [35,41,29,31,29,9,10,16,22,20,0,10,10],   
# [28,30,7,11,13,17,25,27,10,16,10,0,10],
# [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

# demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100] 
# pickup = [0, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
# capacityOfVehicle = 6000
# numberOfVehicles = 4

# E-n22-k4.vrp = CE22P

# xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
# yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

# costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
# for i in range(len(xCoordinates)):
#     for j in range(len(yCoordinates)):
#         costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

# demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500, 700]
# pickup = [0, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500]
# capacityOfVehicle = 6000
# numberOfVehicles = 7

def route_distance(route):
    """Distance for a route (route = list of customer indices, depot index = 0)."""
    if not route:
        return 0
    d = costMatrix[0][route[0]]
    for a, b in zip(route, route[1:]):
        d += costMatrix[a][b]
    d += costMatrix[route[-1]][0]
    return d

def total_delivery(route):
    return sum(demand[i] for i in route)

def feasibility_check_route(route, Q):
    """
    VRPSPD feasibility via cumulative net-pickup:
      initial load = total deliveries for route (vehicle leaves depot carrying all deliveries)
      traverse route: load = load - delivery + pickup
      check 0 <= load <= Q at every step
    Returns: (feasible: bool, max_load_seen, final_load)
    """
    if not route:
        return True, 0, 0
    Dr = total_delivery(route)
    load = Dr
    if load > Q:
        return False, load, load
    max_load = load
    for i in route:
        load = load - demand[i] + pickup[i]
        if load < 0:
            return False, max_load, load
        if load > max_load:
            max_load = load
        if load > Q:
            return False, max_load, load
    return True, max_load, load

# --- Initialize routes: one per customer ---
customers = list(range(1, len(costMatrix)))  # 1..12
routes = [[i] for i in customers]

# --- Compute savings ---
savings = []
for i in customers:
    for j in customers:
        if j <= i:
            continue
        s = costMatrix[0][i] + costMatrix[0][j] - costMatrix[i][j]
        savings.append(((i, j), s))
savings.sort(key=lambda x: x[1], reverse=True)

def find_route_index(routes, cust):
    for idx, r in enumerate(routes):
        if cust in r:
            return idx
    return None

# --- Process savings ---
for (i, j), s in savings:
    ri = find_route_index(routes, i)
    rj = find_route_index(routes, j)
    if ri is None or rj is None or ri == rj:
        continue
    route_i = routes[ri]
    route_j = routes[rj]
    # endpoints only
    if not (i == route_i[0] or i == route_i[-1]):
        continue
    if not (j == route_j[0] or j == route_j[-1]):
        continue
    new_route = None
    # four possible concatenations to keep endpoints adjacency
    if i == route_i[-1] and j == route_j[0]:
        cand = route_i + route_j
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[0] and j == route_j[-1]:
        cand = route_j + route_i
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[0] and j == route_j[0]:
        cand = list(reversed(route_i)) + route_j
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[-1] and j == route_j[-1]:
        cand = route_i + list(reversed(route_j))
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is not None:
        # merge (remove higher index first)
        if ri > rj:
            del routes[ri]
            del routes[rj]
        else:
            del routes[rj]
            del routes[ri]
        routes.append(new_route)

# --- Output ---
print("Final routes (customer indices):")
total_dist = 0.0
for idx, r in enumerate(routes, 1):
    d = route_distance(r)
    total_dist += d
    feas, max_load, final_load = feasibility_check_route(r, capacityOfVehicle)
    print(f" Route {idx}: {r}  | Distance = {d:.1f} | Feasible = {feas} | Max_load_seen = {max_load} | Total_delivery = {total_delivery(r)}")

print(f"\nNumber of routes: {len(routes)} (Vehicles available: {numberOfVehicles})")
print(f"Total distance (all routes): {total_dist:.1f}")
if len(routes) > numberOfVehicles:
    print("WARNING: result needs more vehicles than available!")

print("\nRoutes with depot (0) shown:")
for idx, r in enumerate(routes, 1):
    print(f" Route {idx}: {[0] + r + [0]}")


Final routes (customer indices):
 Route 1: [12, 15, 45, 33, 39, 30, 34, 21, 29, 20, 35, 36, 3, 28, 31, 26, 8]  | Distance = 197.7 | Feasible = True | Max_load_seen = 160 | Total_delivery = 160
 Route 2: [6, 27, 32, 1, 22, 2, 16, 50, 9, 10, 49, 38, 5, 11]  | Distance = 146.9 | Feasible = True | Max_load_seen = 144 | Total_delivery = 142
 Route 3: [46, 47, 18, 4, 17, 37, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48]  | Distance = 206.1 | Feasible = True | Max_load_seen = 160 | Total_delivery = 133

Number of routes: 3 (Vehicles available: 3)
Total distance (all routes): 550.7

Routes with depot (0) shown:
 Route 1: [0, 12, 15, 45, 33, 39, 30, 34, 21, 29, 20, 35, 36, 3, 28, 31, 26, 8, 0]
 Route 2: [0, 6, 27, 32, 1, 22, 2, 16, 50, 9, 10, 49, 38, 5, 11, 0]
 Route 3: [0, 46, 47, 18, 4, 17, 37, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 0]


In [66]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import MDS
# import numpy as np
# import random

# # --- TẠO TỌA ĐỘ TƯỢNG TRƯNG BẰNG MDS ---
# # MDS sẽ cố gắng sắp xếp các điểm trên không gian 2D sao cho
# # khoảng cách Euclide giữa chúng gần nhất với giá trị trong costMatrix.
# print("Đang tạo tọa độ 2D tượng trưng từ costMatrix bằng MDS...")
# mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42, normalized_stress=False)
# # Đảm bảo costMatrix là một numpy array
# node_coordinates = mds.fit_transform(np.array(costMatrix))
# print("Đã tạo xong tọa độ.")

# # --- VẼ BIỂU ĐỒ ---
# plt.style.use('seaborn-v0_8-whitegrid')
# fig, ax = plt.subplots(figsize=(14, 14))

# # 1. Vẽ tất cả các điểm khách hàng
# # Bỏ qua điểm 0 (kho) để vẽ riêng
# customer_coords = node_coordinates[1:]
# ax.scatter(customer_coords[:, 0], customer_coords[:, 1], c='lightblue', s=150, label='Khách hàng', edgecolors='black')

# # 2. Vẽ điểm kho (depot)
# depot_coord = node_coordinates[0]
# ax.scatter(depot_coord[0], depot_coord[1], c='red', s=300, marker='s', label='Kho (Depot)', edgecolors='black')

# # 3. Ghi nhãn cho các điểm
# for i, (x, y) in enumerate(node_coordinates):
#     ax.text(x, y + 1.5, str(i), ha='center', va='bottom', fontweight='bold')

# # 4. Vẽ các tuyến đường
# # 'routes' là biến kết quả từ thuật toán Savings của bạn
# for route in routes:
#     # Tạo một màu ngẫu nhiên cho mỗi tuyến
#     route_color = (random.random(), random.random(), random.random())
    
#     # Tạo danh sách các điểm cần đi qua cho tuyến này, bao gồm cả kho
#     route_path_indices = [0] + route + [0]
    
#     # Lấy tọa độ tương ứng
#     route_path_coords = node_coordinates[route_path_indices]
    
#     # Vẽ các mũi tên chỉ hướng
#     for i in range(len(route_path_coords) - 1):
#         start_point = route_path_coords[i]
#         end_point = route_path_coords[i+1]
#         ax.arrow(start_point[0], start_point[1], 
#                  end_point[0] - start_point[0], end_point[1] - start_point[1],
#                  head_width=1.5, head_length=2, fc=route_color, ec=route_color, length_includes_head=True,
#                  alpha=0.7, lw=1.5)

# # --- TÙY CHỈNH BIỂU ĐỒ ---
# ax.set_title('Biểu đồ các tuyến đường (Tọa độ từ MDS)', fontsize=18)
# ax.set_xlabel('Tọa độ X (tượng trưng)')
# ax.set_ylabel('Tọa độ Y (tượng trưng)')
# ax.legend()
# ax.grid(True)
# ax.set_aspect('equal', adjustable='box') # Đảm bảo tỷ lệ x và y bằng nhau

# plt.show()

In [67]:
# Giả sử 'routes' là biến chứa kết quả từ thuật toán Savings của bạn
# Ví dụ, sau khi chạy code của bạn, 'routes' có thể trông như thế này:
# routes = [[1, 9, 12, 11, 6], [2, 3, 4, 10], [5, 7, 8]]

def convert_routes_to_vector(routes):
    """
    Chuyển đổi một danh sách các tuyến đường sang dạng biểu diễn vector duy nhất.
    Số 0 được sử dụng làm dấu phân cách giữa các tuyến.
    
    Args:
        routes (list of lists): Danh sách các tuyến đường, ví dụ: [[1, 2], [3, 4]]
        
    Returns:
        list: Một vector duy nhất biểu diễn lời giải, ví dụ: [0, 1, 2, 0, 3, 4, 0]
    """
    solution_vector = [0] # Bắt đầu từ kho
    for route in routes:
        solution_vector.extend(route)
        solution_vector.append(0) # Thêm kho để kết thúc tuyến và bắt đầu tuyến mới
    
    # Loại bỏ số 0 cuối cùng nếu nó không cần thiết 
    # (vì tuyến cuối cùng cũng quay về kho)
    # Tuy nhiên, theo Hình 4.5 của luận văn, vector bao gồm cả số 0 cuối cùng.
    # x = [0 1 2 3 4 0 5 6 7 8 9 0]
    # Vì vậy, chúng ta sẽ giữ lại số 0 cuối cùng.
    # Nếu muốn loại bỏ, bạn có thể thêm: if solution_vector[-1] == 0: solution_vector.pop()
    
    return solution_vector

# --- Sử dụng hàm ---

# 'routes' là biến kết quả từ cell code thuật toán Savings của bạn.
# Chúng ta sẽ sử dụng nó ở đây.
initial_solution_vector = convert_routes_to_vector(routes)

print("Kết quả từ thuật toán Savings (danh sách các tuyến):")
print(routes)
print("\nLời giải đã được chuyển đổi sang dạng vector (Solution Representation):")
print(initial_solution_vector)

# Bạn có thể kiểm tra xem nó có hoạt động với một ví dụ khác không
test_routes = [[1, 9, 12], [2, 3, 4, 10], [5, 7, 8]]
test_vector = convert_routes_to_vector(test_routes)
print("\nVí dụ thử nghiệm:")
print(f"  Input: {test_routes}")
print(f"  Output: {test_vector}")

Kết quả từ thuật toán Savings (danh sách các tuyến):
[[12, 15, 45, 33, 39, 30, 34, 21, 29, 20, 35, 36, 3, 28, 31, 26, 8], [6, 27, 32, 1, 22, 2, 16, 50, 9, 10, 49, 38, 5, 11], [46, 47, 18, 4, 17, 37, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48]]

Lời giải đã được chuyển đổi sang dạng vector (Solution Representation):
[0, 12, 15, 45, 33, 39, 30, 34, 21, 29, 20, 35, 36, 3, 28, 31, 26, 8, 0, 6, 27, 32, 1, 22, 2, 16, 50, 9, 10, 49, 38, 5, 11, 0, 46, 47, 18, 4, 17, 37, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 0]

Ví dụ thử nghiệm:
  Input: [[1, 9, 12], [2, 3, 4, 10], [5, 7, 8]]
  Output: [0, 1, 9, 12, 0, 2, 3, 4, 10, 0, 5, 7, 8, 0]


In [68]:
import random

# --- CÁC HÀM PHỤ TRỢ ---
# Hàm này bạn đã có từ bước trước
def convert_routes_to_vector(routes):
    solution_vector = [0]
    for route in routes:
        solution_vector.extend(route)
        solution_vector.append(0)
    return solution_vector

# Hàm này để chuyển đổi ngược lại, rất cần thiết cho việc implement các nước đi
def convert_vector_to_routes(vector):
    """
    Chuyển đổi một vector lời giải về dạng danh sách các tuyến đường.
    """
    routes = []
    current_route = []
    # Bỏ qua số 0 đầu tiên
    for node in vector[1:]:
        if node != 0:
            current_route.append(node)
        else:
            if current_route: # Chỉ thêm nếu tuyến không rỗng
                routes.append(current_route)
            current_route = []
    return routes

In [69]:
# --- IMPLEMENT CÁC "NƯỚC ĐI" (NEIGHBORHOOD MOVES) ---

def general_swap_intra_route(solution_vector, route_idx, i, j):
    """
    Thực hiện nước đi General Swap: Đổi chỗ 2 khách hàng ở vị trí i và j trong cùng 1 tuyến.
    
    Args:
        solution_vector (list): Lời giải hiện tại dạng vector.
        route_idx (int): Chỉ số của tuyến đường cần thay đổi (bắt đầu từ 0).
        i (int): Vị trí của khách hàng thứ nhất trong tuyến.
        j (int): Vị trí của khách hàng thứ hai trong tuyến.
        
    Returns:
        list: Vector lời giải mới (hàng xóm). Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    # Kiểm tra tính hợp lệ
    if not (0 <= route_idx < len(routes)):
        print(f"Lỗi: route_idx {route_idx} không hợp lệ.")
        return None
    
    target_route = routes[route_idx]
    if not (0 <= i < len(target_route) and 0 <= j < len(target_route)):
        print(f"Lỗi: Vị trí i={i} hoặc j={j} không hợp lệ cho tuyến có độ dài {len(target_route)}.")
        return None

    # Thực hiện đổi chỗ
    target_route[i], target_route[j] = target_route[j], target_route[i]
    
    return convert_routes_to_vector(routes)


def relocate_intra_route(solution_vector, route_idx, i, j):
    """
    Thực hiện nước đi Relocate (Single Insertion): Di chuyển khách hàng ở vị trí i
    đến vị trí j trong cùng một tuyến.
    
    Args:
        solution_vector (list): Lời giải hiện tại dạng vector.
        route_idx (int): Chỉ số của tuyến đường cần thay đổi.
        i (int): Vị trí của khách hàng cần di chuyển.
        j (int): Vị trí mới để chèn khách hàng vào.
        
    Returns:
        list: Vector lời giải mới (hàng xóm). Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    if not (0 <= route_idx < len(routes)):
        print(f"Lỗi: route_idx {route_idx} không hợp lệ.")
        return None
        
    target_route = routes[route_idx]
    if not (0 <= i < len(target_route) and 0 <= j <= len(target_route)):
        print(f"Lỗi: Vị trí i={i} hoặc j={j} không hợp lệ.")
        return None

    # Lấy khách hàng ra khỏi vị trí i và chèn vào vị trí j
    customer_to_move = target_route.pop(i)
    target_route.insert(j, customer_to_move)
    
    return convert_routes_to_vector(routes)


def relocate_inter_route(solution_vector, route_idx_from, route_idx_to, i, j):
    """
    Thực hiện nước đi Relocate (Shift): Di chuyển khách hàng ở vị trí i từ tuyến 'from'
    sang vị trí j ở tuyến 'to'.
    
    Args:
        solution_vector (list): Lời giải hiện tại dạng vector.
        route_idx_from (int): Chỉ số của tuyến nguồn.
        route_idx_to (int): Chỉ số của tuyến đích.
        i (int): Vị trí của khách hàng cần di chuyển trong tuyến nguồn.
        j (int): Vị trí mới để chèn khách hàng vào trong tuyến đích.
        
    Returns:
        list: Vector lời giải mới (hàng xóm). Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    if not (0 <= route_idx_from < len(routes) and 0 <= route_idx_to < len(routes)):
        print("Lỗi: Chỉ số tuyến không hợp lệ.")
        return None
        
    route_from = routes[route_idx_from]
    route_to = routes[route_idx_to]

    if len(route_from) <= 1:
        return None
    
    if not (0 <= i < len(route_from) and 0 <= j <= len(route_to)):
        print("Lỗi: Vị trí khách hàng không hợp lệ.")
        return None

    # Di chuyển khách hàng
    customer_to_move = route_from.pop(i)
    route_to.insert(j, customer_to_move)
    
    # Nếu tuyến nguồn trở nên rỗng, hãy xóa nó đi
    if not route_from:
        routes.pop(route_idx_from)
        
    return convert_routes_to_vector(routes)

# # --- SỬ DỤNG VÀ THỬ NGHIỆM ---


In [70]:
# --- HÀM KIỂM TRA TẢI TRỌNG ---

def check_single_route_feasibility(route, Q):
    """
    Kiểm tra tính hợp lệ về tải trọng cho một tuyến đường DUY NHẤT.
    
    Args:
        route (list): Danh sách các khách hàng trong một tuyến (không bao gồm kho).
        Q (int): Tải trọng của xe.
        
    Returns:
        bool: True nếu tuyến đường hợp lệ, False nếu ngược lại.
    """
    if not route:
        return True # Tuyến rỗng luôn hợp lệ
        
    # 1. Kiểm tra sơ bộ
    total_d = sum(demand[c] for c in route)
    total_p = sum(pickup[c] for c in route)
    
    # Tổng lượng hàng phải giao phải nằm trong xe khi xuất phát
    # Tổng lượng hàng nhận về cuối cùng cũng phải nằm trong xe
    if total_d > Q or total_p > Q:
        return False
        
    # 2. Kiểm tra chi tiết tải trọng tại mỗi điểm
    load = total_d
    for customer in route:
        load = load - demand[customer] + pickup[customer]
        # Tải trọng không được vượt quá capacité tại bất kỳ điểm nào
        if not (0 <= load <= Q):
            return False
            
    return True

def feasibility_check_and_repair(routes, Q):
    """
    Kiểm tra tính hợp lệ của tất cả các tuyến đường trong một lời giải.
    Nếu một tuyến không hợp lệ, thử đảo ngược nó.
    
    Args:
        routes (list of lists): Lời giải dưới dạng danh sách các tuyến đường.
        Q (int): Tải trọng của xe.
        
    Returns:
        (bool, list of lists): Một tuple gồm (True/False, danh sách các tuyến đã được sửa chữa nếu có).
                                 Trả về False nếu có ít nhất một tuyến không hợp lệ kể cả sau khi đảo ngược.
    """
    repaired_routes = []
    for route in routes:
        # Kiểm tra theo chiều xuôi
        if check_single_route_feasibility(route, Q):
            repaired_routes.append(route)
            continue # Tuyến này ổn, chuyển sang tuyến tiếp theo
            
        # Nếu chiều xuôi không hợp lệ, thử đảo ngược
        reversed_route = list(reversed(route))
        if check_single_route_feasibility(reversed_route, Q):
            repaired_routes.append(reversed_route)
            continue # Tuyến đảo ngược ổn, chuyển sang tuyến tiếp theo
        
        # Nếu cả hai chiều đều không hợp lệ
        return (False, None)
        
    return (True, repaired_routes)


# # --- CÁCH SỬ DỤNG VÀ THỬ NGHIỆM ---

# # Ví dụ 1: Một tuyến đường hợp lệ
# route_A = [1, 2] 
# # Total demand = 1200 + 1700 = 2900 <= 6000
# # Load ban đầu = 2900
# # Sau điểm 1: load = 2900 - 1200 (demand) + 0 (pickup) = 1700.  (0 <= 1700 <= 6000 -> OK)
# # Sau điểm 2: load = 1700 - 1700 (demand) + 1200 (pickup) = 1200. (0 <= 1200 <= 6000 -> OK)
# is_feasible_A = check_single_route_feasibility(route_A, capacityOfVehicle)
# print(f"Tuyến A {route_A} có hợp lệ không? -> {is_feasible_A}\n") # Sẽ là True


# # Ví dụ 2: Một tuyến đường không hợp lệ (vượt tải trọng giữa chừng)
# route_B = [8, 9, 10, 11]
# # Total demand = 1900 + 1800 + 1600 + 1700 = 7000 > 6000 -> Không hợp lệ ngay từ đầu
# is_feasible_B = check_single_route_feasibility(route_B, capacityOfVehicle)
# print(f"Tuyến B {route_B} có hợp lệ không? -> {is_feasible_B}\n") # Sẽ là False

# # Ví dụ 3: Một tuyến chỉ hợp lệ khi đảo ngược (Giả định dữ liệu khác)
# demand_special = [0, 10, 40]
# pickup_special = [0, 50, 5]
# Q_special = 50

# route_C_fwd = [1, 2] # Chiều xuôi
# # Total demand = 10 + 40 = 50. Load ban đầu = 50.
# # Sau điểm 1: load = 50 - 10 + 50 = 90. (90 > 50 -> KHÔNG HỢP LỆ)

# route_C_rev = [2, 1] # Chiều ngược
# # Total demand = 40 + 10 = 50. Load ban đầu = 50.
# # Sau điểm 2: load = 50 - 40 + 5 = 15. (0 <= 15 <= 50 -> OK)
# # Sau điểm 1: load = 15 - 10 + 50 = 55. (55 > 50 -> VẪN KHÔNG HỢP LỆ)
# # -> Ví dụ này không hợp lệ cả hai chiều. Nhưng nó minh họa logic kiểm tra.

# # Sử dụng hàm tổng hợp `feasibility_check_and_repair`
# solution_routes = [[1, 2], [5, 6]]
# is_sol_feasible, repaired = feasibility_check_and_repair(solution_routes, capacityOfVehicle)
# print(f"Toàn bộ lời giải {solution_routes} có hợp lệ không? -> {is_sol_feasible}")
# if is_sol_feasible:
#     print(f"Lời giải sau khi có thể đã sửa chữa: {repaired}\n")

# # Ví dụ một lời giải không hợp lệ
# solution_routes_bad = [[1, 2], [8, 9, 10, 11]]
# is_sol_feasible_bad, _ = feasibility_check_and_repair(solution_routes_bad, capacityOfVehicle)
# print(f"Toàn bộ lời giải {solution_routes_bad} có hợp lệ không? -> {is_sol_feasible_bad}")

In [71]:
# CELL MỚI: IMPLEMENT CÁC NƯỚC ĐI BỔ SUNG

def swap_one_one_inter_route(solution_vector, route_idx_A, route_idx_B, cust_idx_A, cust_idx_B):
    """
    Thực hiện nước đi Swap(1,1): tráo đổi khách hàng ở vị trí cust_idx_A của tuyến A
    với khách hàng ở vị trí cust_idx_B của tuyến B.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx_A (int): Chỉ số của tuyến thứ nhất.
        route_idx_B (int): Chỉ số của tuyến thứ hai.
        cust_idx_A (int): Vị trí của khách hàng trong tuyến A.
        cust_idx_B (int): Vị trí của khách hàng trong tuyến B.
        
    Returns:
        list: Vector lời giải mới. Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    # Kiểm tra tính hợp lệ
    if not (0 <= route_idx_A < len(routes) and 0 <= route_idx_B < len(routes)):
        return None
    
    route_A = routes[route_idx_A]
    route_B = routes[route_idx_B]
    
    if not (0 <= cust_idx_A < len(route_A) and 0 <= cust_idx_B < len(route_B)):
        return None
        
    # Thực hiện tráo đổi
    route_A[cust_idx_A], route_B[cust_idx_B] = route_B[cust_idx_B], route_A[cust_idx_A]
    
    return convert_routes_to_vector(routes)


def two_opt_intra_route(solution_vector, route_idx, i, j):
    """
    Thực hiện nước đi 2-opt trên một tuyến đường.
    Phá vỡ các cạnh (i, i+1) và (j, j+1), sau đó nối lại.
    Đoạn đường từ (i+1) đến j sẽ bị đảo ngược.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx (int): Chỉ số của tuyến cần áp dụng.
        i (int): Vị trí của khách hàng bắt đầu đoạn đảo ngược.
        j (int): Vị trí của khách hàng kết thúc đoạn đảo ngược.
    
    Returns:
        list: Vector lời giải mới. Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    if not (0 <= route_idx < len(routes)):
        return None
        
    target_route = routes[route_idx]
    
    # i và j là các chỉ số trong tuyến. j phải lớn hơn i.
    if not (0 <= i < j < len(target_route)):
        return None
        
    # Tạo tuyến mới bằng cách ghép các phần
    # Phần đầu: từ đầu đến hết khách hàng i
    part1 = target_route[0 : i+1]
    # Phần giữa: từ khách hàng i+1 đến j, được đảo ngược
    part2_reversed = target_route[i+1 : j+1]
    part2_reversed.reverse()
    # Phần cuối: từ sau khách hàng j đến hết
    part3 = target_route[j+1 : ]
    
    # Ghép lại thành tuyến mới
    new_route = part1 + part2_reversed + part3
    
    # Cập nhật lại danh sách các tuyến
    routes[route_idx] = new_route
    
    return convert_routes_to_vector(routes)

# # --- Thử nghiệm nhanh các hàm mới ---
# test_vector_for_new_moves = [0, 10, 20, 30, 40, 50, 0, 60, 70, 80, 0]
# print(f"Vector thử nghiệm ban đầu:\n{test_vector_for_new_moves}\n")

# print("--- Thử nghiệm Swap(1,1) ---")
# # Tráo đổi khách hàng 20 (vị trí 1, tuyến 0) với khách hàng 70 (vị trí 1, tuyến 1)
# swapped_vector = swap_one_one_inter_route(test_vector_for_new_moves, 0, 1, 1, 1)
# print(f"Vector sau khi tráo đổi: \n{swapped_vector}\n")

# print("--- Thử nghiệm 2-opt ---")
# # Áp dụng 2-opt cho tuyến đầu tiên [10, 20, 30, 40, 50]
# # Đảo ngược đoạn từ vị trí 1 (20) đến vị trí 3 (40)
# # Kết quả mong đợi: [10] + [40, 30, 20] + [50] = [10, 40, 30, 20, 50]
# two_opt_vector = two_opt_intra_route(test_vector_for_new_moves, 0, i=0, j=3)
# print(f"Vector sau khi 2-opt: \n{two_opt_vector}\n")

In [72]:
# CELL MỚI: IMPLEMENT CÁC NƯỚC ĐI KHỐI (BLOCK MOVES)

def shift_two_zero_inter_route(solution_vector, route_idx_from, route_idx_to, block_start_idx, insert_idx):
    """
    Thực hiện nước đi Shift(2,0): Di chuyển một khối 2 khách hàng liền kề từ
    tuyến 'from' sang tuyến 'to'.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx_from (int): Chỉ số của tuyến nguồn.
        route_idx_to (int): Chỉ số của tuyến đích.
        block_start_idx (int): Vị trí bắt đầu của khối trong tuyến nguồn.
        insert_idx (int): Vị trí để chèn khối vào trong tuyến đích.
        
    Returns:
        list: Vector lời giải mới. Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    # --- Kiểm tra tính hợp lệ của các chỉ số ---
    if not (0 <= route_idx_from < len(routes) and 0 <= route_idx_to < len(routes) and route_idx_from != route_idx_to):
        return None
        
    route_from = routes[route_idx_from]
    route_to = routes[route_idx_to]
    
    # Tuyến nguồn phải có đủ 2 khách hàng để tạo thành khối
    if len(route_from) < 2 or not (0 <= block_start_idx < len(route_from) - 1):
        return None
    
    # Vị trí chèn phải hợp lệ
    if not (0 <= insert_idx <= len(route_to)):
        return None

    # --- Thực hiện nước đi ---
    # Lấy khối 2 khách hàng ra
    block_to_move = route_from[block_start_idx : block_start_idx + 2]
    
    # Xóa khối khỏi tuyến nguồn
    del route_from[block_start_idx : block_start_idx + 2]
    
    # Chèn khối vào tuyến đích
    # Cách 1: Dùng list.insert nhiều lần
    # route_to.insert(insert_idx, block_to_move[1])
    # route_to.insert(insert_idx, block_to_move[0])
    # Cách 2: Dùng slicing (dễ đọc hơn)
    new_route_to = route_to[:insert_idx] + block_to_move + route_to[insert_idx:]
    routes[route_idx_to] = new_route_to

    # Nếu tuyến nguồn trở nên rỗng, hãy xóa nó đi
    # Cần cẩn thận khi xóa phần tử khỏi list đang được duyệt chỉ số
    if not route_from:
        # Xóa tuyến rỗng và tạo lại vector từ danh sách tuyến đã cập nhật
        final_routes = [r for r in routes if r]
        return convert_routes_to_vector(final_routes)
    else:
        # Cập nhật lại tuyến nguồn đã bị thay đổi
        routes[route_idx_from] = route_from
        return convert_routes_to_vector(routes)


def swap_two_one_inter_route(solution_vector, route_idx_A, route_idx_B, block_start_idx_A, cust_idx_B):
    """
    Thực hiện nước đi Swap(2,1): Tráo đổi khối 2 khách hàng từ tuyến A
    với 1 khách hàng từ tuyến B.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx_A (int): Chỉ số tuyến có khối.
        route_idx_B (int): Chỉ số tuyến có khách hàng đơn.
        block_start_idx_A (int): Vị trí bắt đầu khối trong tuyến A.
        cust_idx_B (int): Vị trí của khách hàng trong tuyến B.
        
    Returns:
        list: Vector lời giải mới. Trả về None nếu đầu vào không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    # --- Kiểm tra tính hợp lệ ---
    if not (0 <= route_idx_A < len(routes) and 0 <= route_idx_B < len(routes) and route_idx_A != route_idx_B):
        return None
        
    route_A = routes[route_idx_A]
    route_B = routes[route_idx_B]
    
    if len(route_A) < 2 or not (0 <= block_start_idx_A < len(route_A) - 1):
        return None
    
    if not (0 <= cust_idx_B < len(route_B)):
        return None

    # --- Thực hiện nước đi ---
    # Lưu lại các phần tử cần tráo đổi
    block_from_A = route_A[block_start_idx_A : block_start_idx_A + 2]
    customer_from_B = route_B[cust_idx_B]
    
    # Xây dựng lại tuyến A: thay thế khối bằng khách hàng đơn
    new_route_A = route_A[:block_start_idx_A] + [customer_from_B] + route_A[block_start_idx_A + 2:]
    
    # Xây dựng lại tuyến B: thay thế khách hàng đơn bằng khối
    new_route_B = route_B[:cust_idx_B] + block_from_A + route_B[cust_idx_B + 1:]
    
    # Cập nhật lại danh sách các tuyến
    routes[route_idx_A] = new_route_A
    routes[route_idx_B] = new_route_B
    
    return convert_routes_to_vector(routes)


# # --- Thử nghiệm nhanh các hàm mới ---
# test_vector_block_moves = [0, 10, 20, 30, 0, 40, 50, 0]
# print(f"Vector thử nghiệm ban đầu:\n{test_vector_block_moves}\n")

# print("--- Thử nghiệm Shift(2,0) ---")
# # Di chuyển khối [10, 20] từ tuyến 0 (idx 0) sang vị trí đầu tiên (idx 0) của tuyến 1
# shifted_vector = shift_two_zero_inter_route(test_vector_block_moves, 0, 1, 0, 0)
# # Tuyến 0 còn lại: [30]. Tuyến 1 mới: [10, 20, 40, 50]
# # Vector mong đợi: [0, 30, 0, 10, 20, 40, 50, 0]
# print(f"Vector sau khi di chuyển khối: \n{shifted_vector}\n")

# print("--- Thử nghiệm Swap(2,1) ---")
# # Tráo đổi khối [20, 30] từ tuyến 0 (idx 0) với khách hàng 40 từ tuyến 1 (idx 1)
# swapped_vector_2_1 = swap_two_one_inter_route(test_vector_block_moves, 0, 1, 1, 0)
# # Tuyến 0 mới: [10, 40]. Tuyến 1 mới: [20, 30, 50]
# # Vector mong đợi: [0, 10, 40, 0, 20, 30, 50, 0]
# print(f"Vector sau khi tráo đổi khối-đơn: \n{swapped_vector_2_1}\n")

In [73]:
# CELL MỚI: IMPLEMENT CÁC NƯỚC ĐI KHỐI CUỐI CÙNG

def block_insertion_intra_route(solution_vector, route_idx, block_start_idx, insert_idx):
    """
    Thực hiện nước đi Block Insertion: Di chuyển một khối 2 khách hàng liền kề
    đến một vị trí mới trong cùng một tuyến.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx (int): Chỉ số của tuyến cần áp dụng.
        block_start_idx (int): Vị trí bắt đầu của khối.
        insert_idx (int): Vị trí mới để chèn khối vào.
        
    Returns:
        list: Vector lời giải mới. Trả về None nếu không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    if not (0 <= route_idx < len(routes)): return None
        
    target_route = routes[route_idx]
    
    # Tuyến phải đủ dài để chứa một khối và còn chỗ để di chuyển
    if len(target_route) < 3: return None
    if not (0 <= block_start_idx < len(target_route) - 1): return None

    # Lấy khối ra
    block_to_move = target_route[block_start_idx : block_start_idx + 2]
    
    # Tạo một tuyến tạm thời đã xóa khối
    temp_route = target_route[:block_start_idx] + target_route[block_start_idx + 2:]
    
    # Vị trí chèn phải hợp lệ trên tuyến tạm thời
    if not (0 <= insert_idx <= len(temp_route)): return None
        
    # Chèn khối vào vị trí mới để tạo tuyến đã được cập nhật
    new_route = temp_route[:insert_idx] + block_to_move + temp_route[insert_idx:]
    
    routes[route_idx] = new_route
    return convert_routes_to_vector(routes)


def swap_two_two_inter_route(solution_vector, route_idx_A, route_idx_B, block_start_idx_A, block_start_idx_B):
    """
    Thực hiện nước đi Swap(2,2): Tráo đổi một khối 2 khách hàng từ tuyến A
    với một khối 2 khách hàng từ tuyến B.
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        route_idx_A (int): Chỉ số tuyến A.
        route_idx_B (int): Chỉ số tuyến B.
        block_start_idx_A (int): Vị trí bắt đầu khối trong tuyến A.
        block_start_idx_B (int): Vị trí bắt đầu khối trong tuyến B.
        
    Returns:
        list: Vector lời giải mới. Trả về None nếu không hợp lệ.
    """
    routes = convert_vector_to_routes(solution_vector)
    
    if not (0 <= route_idx_A < len(routes) and 0 <= route_idx_B < len(routes) and route_idx_A != route_idx_B):
        return None
        
    route_A = routes[route_idx_A]
    route_B = routes[route_idx_B]
    
    if len(route_A) < 2 or not (0 <= block_start_idx_A < len(route_A) - 1):
        return None
        
    if len(route_B) < 2 or not (0 <= block_start_idx_B < len(route_B) - 1):
        return None

    # Lưu lại các khối cần tráo đổi
    block_from_A = route_A[block_start_idx_A : block_start_idx_A + 2]
    block_from_B = route_B[block_start_idx_B : block_start_idx_B + 2]
    
    # Xây dựng lại tuyến A: thay khối A bằng khối B
    new_route_A = route_A[:block_start_idx_A] + block_from_B + route_A[block_start_idx_A + 2:]
    
    # Xây dựng lại tuyến B: thay khối B bằng khối A
    new_route_B = route_B[:block_start_idx_B] + block_from_A + route_B[block_start_idx_B + 1:]
    
    routes[route_idx_A] = new_route_A
    routes[route_idx_B] = new_route_B
    
    return convert_routes_to_vector(routes)


# # --- Thử nghiệm nhanh các hàm mới ---
# test_vector_final_moves = [0, 10, 20, 30, 40, 50, 0, 60, 70, 80, 90, 0]
# print(f"Vector thử nghiệm ban đầu:\n{test_vector_final_moves}\n")

# print("--- Thử nghiệm Block Insertion (intra-route) ---")
# # Di chuyển khối [20, 30] từ vị trí 1 của tuyến 0 sang vị trí 3
# # Tuyến 0 ban đầu: [10, 20, 30, 40, 50]
# # Tuyến 0 mới mong đợi: [10, 40, 50, 20, 30]
# block_inserted_vector = block_insertion_intra_route(test_vector_final_moves, 0, 1, 3)
# print(f"Vector sau khi chèn khối: \n{block_inserted_vector}\n")

# print("--- Thử nghiệm Swap(2,2) ---")
# # Tráo đổi khối [20, 30] từ tuyến 0 với khối [70, 80] từ tuyến 1
# swapped_2_2_vector = swap_two_two_inter_route(test_vector_final_moves, 0, 1, 1, 1)
# # Tuyến 0 mới: [10, 70, 80, 40, 50]
# # Tuyến 1 mới: [60, 20, 30, 90]
# print(f"Vector sau khi tráo đổi khối-khối: \n{swapped_2_2_vector}\n")

In [74]:
# CELL MỚI: HÀM TÌM KIẾM CỤC BỘ TRONG TUYẾN (INTRA-ROUTE VND)

def vnd_intra_search(solution_vector):
    """
    Áp dụng VND cục bộ trên từng tuyến của một lời giải để cải tiến sâu (intensify).
    Hàm này chỉ sử dụng các nước đi intra-route.
    
    Args:
        solution_vector (list): Vector lời giải cần cải tiến.
        
    Returns:
        list: Vector lời giải đã được cải tiến cục bộ.
    """
    
    current_routes = convert_vector_to_routes(solution_vector)
    improved_routes = []

    intra_neighborhoods = [
        # Danh sách các nước đi intra-route, từ đơn giản đến phức tạp
        lambda r, i, j: (r[:i] + [r[j]] + r[i+1:j] + [r[i]] + r[j+1:]), # General Swap
        lambda r, i, j: (r[:i] + r[i+1:j] + [r[i]] + r[j:]), # Relocate (di chuyển i tới trước j)
        lambda r, i, j: (r[:i] + r[i+2:j] + [r[i], r[i+1]] + r[j:]), # Block Insertion (chèn khối bắt đầu từ i tới trước j)
        lambda r, i, j: (r[:i+1] + r[i+1:j+1][::-1] + r[j+1:]), # 2-Opt
    ]
    
    for route in current_routes:
        if len(route) <= 1: # Không thể cải thiện tuyến có 0 hoặc 1 khách
            improved_routes.append(route)
            continue

        best_route = route
        best_route_cost = route_distance(best_route) # Dùng lại hàm tính chi phí một tuyến

        k = 0
        while k < len(intra_neighborhoods):
            improvement_found = False
            
            # Khám phá tất cả các hàng xóm có thể có của nước đi loại k
            # (Các generator này được viết gọn lại cho hiệu quả)
            if k == 0: # General Swap
                neighbors_gen = (intra_neighborhoods[k](best_route, i, j) for i in range(len(best_route)) for j in range(i + 1, len(best_route)))
            elif k == 1: # Relocate
                neighbors_gen = (intra_neighborhoods[k](best_route, i, j) for i in range(len(best_route)) for j in range(len(best_route) + 1) if i != j and i != j-1)
            elif k == 2: # Block Insertion
                if len(best_route) < 3: neighbors_gen = iter([]) # Bỏ qua nếu tuyến quá ngắn
                else: neighbors_gen = (intra_neighborhoods[k](best_route, i, j) for i in range(len(best_route)-1) for j in range(len(best_route)-1) if j != i and j != i+1)
            elif k == 3: # 2-Opt
                if len(best_route) < 3: neighbors_gen = iter([]) # Bỏ qua nếu tuyến quá ngắn
                else: neighbors_gen = (intra_neighborhoods[k](best_route, i, j) for i in range(len(best_route) - 1) for j in range(i + 2, len(best_route)))
            
            for neighbor_route in neighbors_gen:
                is_feasible, _ = feasibility_check_and_repair([neighbor_route], capacityOfVehicle)
                
                if is_feasible:
                    neighbor_cost = route_distance(neighbor_route)
                    if neighbor_cost < best_route_cost:
                        best_route = neighbor_route
                        best_route_cost = neighbor_cost
                        improvement_found = True
                        break # First improvement
            
            if improvement_found:
                k = 0 # Quay lại nước đi đơn giản nhất
            else:
                k += 1 # Chuyển sang nước đi tiếp theo
        
        improved_routes.append(best_route)
        
    return convert_routes_to_vector(improved_routes)

In [75]:
import time
import math # Để dùng cho math.inf

# --- Đảm bảo các hàm từ các bước trước đã tồn tại ---
# (Bạn chỉ cần chạy các cell trước đó trong notebook)
# - costMatrix, demand, pickup, capacityOfVehicle
# - convert_routes_to_vector, convert_vector_to_routes
# - general_swap_intra_route, relocate_intra_route, relocate_inter_route
# - feasibility_check_and_repair
# - initial_solution_vector (kết quả từ thuật toán Savings)

# --- HÀM TÍNH TỔNG CHI PHÍ ---
def calculate_total_cost(solution_vector):
    """
    Tính tổng chi phí (quãng đường) cho một lời giải dạng vector.
    """
    routes = convert_vector_to_routes(solution_vector)
    total_cost = 0
    for route in routes:
        if not route:
            continue
        # Chi phí từ kho đến điểm đầu tiên
        total_cost += costMatrix[0][route[0]]
        # Chi phí giữa các khách hàng
        for i in range(len(route) - 1):
            total_cost += costMatrix[route[i]][route[i+1]]
        # Chi phí từ điểm cuối cùng về kho
        total_cost += costMatrix[route[-1]][0]
    return total_cost

def variable_neighborhood_descent_two_level(initial_vector, max_time_seconds=60):
    """
    Thực hiện thuật toán VND hai cấp (Inter-Intra) để cải thiện lời giải.
    Phiên bản này tuân thủ chặt chẽ với giả mã p_VND trong luận văn.
    """
    start_time = time.time()
    
    print("--- Bắt đầu VND hai cấp (phiên bản tuân thủ p_VND) ---")
    
    # 1. Khởi tạo (KHÔNG có bước tiền xử lý vnd_intra_search)
    current_vector = initial_vector
    current_cost = calculate_total_cost(current_vector)
    
    best_vector = current_vector
    best_cost = current_cost
    
    print(f"Chi phí ban đầu: {best_cost:.2f}")

    # 2. Danh sách các nước đi INTER-ROUTE, sắp xếp theo "Ordering 2"
    inter_neighborhoods = [
        # a) Swap(1,1)
        lambda vec: (swap_one_one_inter_route(vec, r_A_idx, r_B_idx, i, j) for r_A_idx, r_A in enumerate(convert_vector_to_routes(vec)) for r_B_idx, r_B in enumerate(convert_vector_to_routes(vec)) if r_A_idx < r_B_idx for i in range(len(r_A)) for j in range(len(r_B))),
        # b) Shift(1,0) - Relocate
        lambda vec: (relocate_inter_route(vec, r_from_idx, r_to_idx, i, j) for r_from_idx, r_from in enumerate(convert_vector_to_routes(vec)) for r_to_idx, r_to in enumerate(convert_vector_to_routes(vec)) if r_from_idx != r_to_idx for i in range(len(r_from)) for j in range(len(r_to) + 1)),
        # c) Shift(2,0)
        lambda vec: (shift_two_zero_inter_route(vec, r_from_idx, r_to_idx, i, j) for r_from_idx, r_from in enumerate(convert_vector_to_routes(vec)) if len(r_from) >= 2 for r_to_idx, r_to in enumerate(convert_vector_to_routes(vec)) if r_from_idx != r_to_idx for i in range(len(r_from) - 1) for j in range(len(r_to) + 1)),
        # d) Swap(2,1)
        lambda vec: (swap_two_one_inter_route(vec, r_A_idx, r_B_idx, i, j) for r_A_idx, r_A in enumerate(convert_vector_to_routes(vec)) if len(r_A) >= 2 for r_B_idx, r_B in enumerate(convert_vector_to_routes(vec)) if r_A_idx != r_B_idx for i in range(len(r_A) - 1) for j in range(len(r_B))),
        # e) Swap(2,2)
        lambda vec: (swap_two_two_inter_route(vec, r_A_idx, r_B_idx, i, j) for r_A_idx, r_A in enumerate(convert_vector_to_routes(vec)) if len(r_A) >= 2 for r_B_idx, r_B in enumerate(convert_vector_to_routes(vec)) if r_A_idx < r_B_idx and len(r_B) >= 2 for i in range(len(r_A) - 1) for j in range(len(r_B) - 1)),
    ]
    move_names = ["Swap(1,1)", "Shift(1,0)", "Shift(2,0)", "Swap(2,1)", "Swap(2,2)"]

    k = 0
    while k < len(inter_neighborhoods) and (time.time() - start_time) < max_time_seconds:
        improvement_found = False
        neighbor_generator = inter_neighborhoods[k](current_vector)
        
        # Biến để lưu trữ lời giải tốt nhất tìm được trong toàn bộ neighborhood này
        best_neighbor_in_k_vector = None
        best_neighbor_in_k_cost = current_cost
        
        for neighbor_vector in neighbor_generator:
            if (time.time() - start_time) > max_time_seconds:
                print("Hết thời gian!")
                break
            if neighbor_vector is None: continue

            neighbor_routes = convert_vector_to_routes(neighbor_vector)
            is_feasible, repaired_routes = feasibility_check_and_repair(neighbor_routes, capacityOfVehicle)
            
            if is_feasible:
                # Tích hợp VND_intra: Gọi hàm cải tiến cục bộ NGAY LẬP TỨC
                intensified_vector = vnd_intra_search(convert_routes_to_vector(repaired_routes))
                intensified_cost = calculate_total_cost(intensified_vector)
                
                # Chiến lược Best Improvement: tìm nước đi tốt nhất trong toàn bộ neighborhood
                if intensified_cost < best_neighbor_in_k_cost:
                    best_neighbor_in_k_vector = intensified_vector
                    best_neighbor_in_k_cost = intensified_cost
                    # Lưu ý: Không 'break' ở đây để tìm hết các hàng xóm

        # Sau khi đã duyệt hết hàng xóm của nước đi k
        if best_neighbor_in_k_vector is not None:
            current_vector = best_neighbor_in_k_vector
            current_cost = best_neighbor_in_k_cost
            print(f"  -> Cải thiện! Chi phí mới: {current_cost:.2f} (dùng nước đi inter: {move_names[k]} + intra search)")
            
            if current_cost < best_cost:
                best_vector = current_vector
                best_cost = current_cost
            
            improvement_found = True
        
        if improvement_found:
            k = 0 # Quay lại nước đi đầu tiên
        else:
            print(f"Không tìm thấy cải thiện nào với nước đi inter-route '{move_names[k]}'. Chuyển sang nước đi tiếp theo.")
            k += 1
            
    end_time = time.time()
    print("\n--- Kết thúc VND hai cấp ---")
    print(f"Thuật toán kết thúc sau {end_time - start_time:.2f} giây.")
    print(f"Chi phí ban đầu (từ Savings): {calculate_total_cost(initial_vector):.2f}")
    print(f"Chi phí tốt nhất tìm được: {best_cost:.2f}")
    
    return best_vector, best_cost

# --- Cách gọi hàm mới ---
final_vector, final_cost = variable_neighborhood_descent_two_level(initial_solution_vector, max_time_seconds=60)

--- Bắt đầu VND hai cấp (phiên bản tuân thủ p_VND) ---
Chi phí ban đầu: 550.66
  -> Cải thiện! Chi phí mới: 536.86 (dùng nước đi inter: Swap(1,1) + intra search)
  -> Cải thiện! Chi phí mới: 531.81 (dùng nước đi inter: Swap(1,1) + intra search)
Hết thời gian!
  -> Cải thiện! Chi phí mới: 528.18 (dùng nước đi inter: Swap(1,1) + intra search)

--- Kết thúc VND hai cấp ---
Thuật toán kết thúc sau 60.01 giây.
Chi phí ban đầu (từ Savings): 550.66
Chi phí tốt nhất tìm được: 528.18


In [76]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import MDS
# import numpy as np
# import random

# # --- TẠO TỌA ĐỘ TƯỢNG TRƯNG BẰNG MDS ---
# # MDS sẽ cố gắng sắp xếp các điểm trên không gian 2D sao cho
# # khoảng cách Euclide giữa chúng gần nhất với giá trị trong costMatrix.
# print("Đang tạo tọa độ 2D tượng trưng từ costMatrix bằng MDS...")
# mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42, normalized_stress=False)
# # Đảm bảo costMatrix là một numpy array
# node_coordinates = mds.fit_transform(np.array(costMatrix))
# print("Đã tạo xong tọa độ.")

# # --- VẼ BIỂU ĐỒ ---
# plt.style.use('seaborn-v0_8-whitegrid')
# fig, ax = plt.subplots(figsize=(14, 14))

# # 1. Vẽ tất cả các điểm khách hàng
# # Bỏ qua điểm 0 (kho) để vẽ riêng
# customer_coords = node_coordinates[1:]
# ax.scatter(customer_coords[:, 0], customer_coords[:, 1], c='lightblue', s=150, label='Khách hàng', edgecolors='black')

# # 2. Vẽ điểm kho (depot)
# depot_coord = node_coordinates[0]
# ax.scatter(depot_coord[0], depot_coord[1], c='red', s=300, marker='s', label='Kho (Depot)', edgecolors='black')

# # 3. Ghi nhãn cho các điểm
# for i, (x, y) in enumerate(node_coordinates):
#     ax.text(x, y + 1.5, str(i), ha='center', va='bottom', fontweight='bold')

# # 4. Vẽ các tuyến đường
# # 'routes' là biến kết quả từ thuật toán Savings của bạn
# for route in routes:
#     # Tạo một màu ngẫu nhiên cho mỗi tuyến
#     route_color = (random.random(), random.random(), random.random())
    
#     # Tạo danh sách các điểm cần đi qua cho tuyến này, bao gồm cả kho
#     route_path_indices = [0] + route + [0]
    
#     # Lấy tọa độ tương ứng
#     route_path_coords = node_coordinates[route_path_indices]
    
#     # Vẽ các mũi tên chỉ hướng
#     for i in range(len(route_path_coords) - 1):
#         start_point = route_path_coords[i]
#         end_point = route_path_coords[i+1]
#         ax.arrow(start_point[0], start_point[1], 
#                  end_point[0] - start_point[0], end_point[1] - start_point[1],
#                  head_width=1.5, head_length=2, fc=route_color, ec=route_color, length_includes_head=True,
#                  alpha=0.7, lw=1.5)

# # --- TÙY CHỈNH BIỂU ĐỒ ---
# ax.set_title('Biểu đồ các tuyến đường (Tọa độ từ MDS)', fontsize=18)
# ax.set_xlabel('Tọa độ X (tượng trưng)')
# ax.set_ylabel('Tọa độ Y (tượng trưng)')
# ax.legend()
# ax.grid(True)
# ax.set_aspect('equal', adjustable='box') # Đảm bảo tỷ lệ x và y bằng nhau

# plt.show()

In [77]:
import random

def get_random_neighbor(solution_vector):
    """
    Chọn ngẫu nhiên một cấu trúc lân cận và sinh ra một lời giải hàng xóm ngẫu nhiên.
    Hàm này thử tối đa 10 lần để tìm một nước đi hợp lệ (feasible).
    
    Args:
        solution_vector (list): Lời giải hiện tại.
        
    Returns:
        (list, float): Vector lời giải hàng xóm và chi phí của nó.
                       Trả về (None, None) nếu không tìm được.
    """
    routes = convert_vector_to_routes(solution_vector)
    num_routes = len(routes)
    
    # Danh sách các hàm nước đi (chúng ta đã define ở các bước trước)
    # Lưu ý: Các hàm này cần nhận tham số đúng định dạng.
    # Vì tham số khác nhau, ta sẽ xử lý logic random bên trong vòng lặp.
    
    # 0: General Swap, 1: Relocate Intra, 2: Relocate Inter, 
    # 3: Swap(1,1), 4: Block Insert, 5: Shift(2,0), 
    # 6: Swap(2,1), 7: Swap(2,2), 8: 2-Opt
    
    for _ in range(10): # Thử tối đa 10 lần để tìm một nước đi feasible
        move_type = random.randint(0, 8)
        new_vector = None
        
        try:
            if move_type == 0: # General Swap (Intra)
                r_idx = random.randint(0, num_routes - 1)
                if len(routes[r_idx]) < 2: continue
                i, j = random.sample(range(len(routes[r_idx])), 2)
                new_vector = general_swap_intra_route(solution_vector, r_idx, i, j)
                
            elif move_type == 1: # Relocate (Intra)
                r_idx = random.randint(0, num_routes - 1)
                if len(routes[r_idx]) < 2: continue
                i = random.randint(0, len(routes[r_idx]) - 1)
                j = random.randint(0, len(routes[r_idx])) # vị trí chèn có thể là cuối
                if i == j or i + 1 == j: continue
                new_vector = relocate_intra_route(solution_vector, r_idx, i, j)
                
            elif move_type == 2: # Relocate (Inter)
                if num_routes < 2: continue
                r_from, r_to = random.sample(range(num_routes), 2)
                if not routes[r_from]: continue
                i = random.randint(0, len(routes[r_from]) - 1)
                j = random.randint(0, len(routes[r_to]))
                new_vector = relocate_inter_route(solution_vector, r_from, r_to, i, j)
                
            elif move_type == 3: # Swap(1,1) (Inter)
                if num_routes < 2: continue
                r_A, r_B = random.sample(range(num_routes), 2)
                if not routes[r_A] or not routes[r_B]: continue
                i = random.randint(0, len(routes[r_A]) - 1)
                j = random.randint(0, len(routes[r_B]) - 1)
                new_vector = swap_one_one_inter_route(solution_vector, r_A, r_B, i, j)
                
            elif move_type == 4: # Block Insertion (Intra)
                r_idx = random.randint(0, num_routes - 1)
                if len(routes[r_idx]) < 3: continue
                # Chọn khối [i, i+1]
                i = random.randint(0, len(routes[r_idx]) - 2)
                # Chọn vị trí chèn j (trên tuyến tạm thời đã bỏ khối)
                # Tuyến tạm có len - 2 phần tử -> vị trí chèn từ 0 đến len - 2
                j = random.randint(0, len(routes[r_idx]) - 2)
                new_vector = block_insertion_intra_route(solution_vector, r_idx, i, j)

            elif move_type == 5: # Shift(2,0) (Inter)
                if num_routes < 2: continue
                r_from, r_to = random.sample(range(num_routes), 2)
                if len(routes[r_from]) < 2: continue
                i = random.randint(0, len(routes[r_from]) - 2) # Start block
                j = random.randint(0, len(routes[r_to])) # Insert pos
                new_vector = shift_two_zero_inter_route(solution_vector, r_from, r_to, i, j)

            elif move_type == 6: # Swap(2,1) (Inter)
                if num_routes < 2: continue
                # Chọn hướng: A(2) <-> B(1) hay A(1) <-> B(2)
                direction = random.choice([0, 1])
                r_A, r_B = random.sample(range(num_routes), 2)
                
                if direction == 0: # A(2), B(1)
                    if len(routes[r_A]) < 2 or not routes[r_B]: continue
                    i = random.randint(0, len(routes[r_A]) - 2)
                    j = random.randint(0, len(routes[r_B]) - 1)
                    new_vector = swap_two_one_inter_route(solution_vector, r_A, r_B, i, j)
                else: # A(1), B(2) -> Gọi hàm ngược lại swap_two_one(B, A, ...)
                    if len(routes[r_B]) < 2 or not routes[r_A]: continue
                    i = random.randint(0, len(routes[r_B]) - 2)
                    j = random.randint(0, len(routes[r_A]) - 1)
                    new_vector = swap_two_one_inter_route(solution_vector, r_B, r_A, i, j)

            elif move_type == 7: # Swap(2,2) (Inter)
                if num_routes < 2: continue
                r_A, r_B = random.sample(range(num_routes), 2)
                if len(routes[r_A]) < 2 or len(routes[r_B]) < 2: continue
                i = random.randint(0, len(routes[r_A]) - 2)
                j = random.randint(0, len(routes[r_B]) - 2)
                new_vector = swap_two_two_inter_route(solution_vector, r_A, r_B, i, j)

            elif move_type == 8: # 2-Opt (Intra)
                r_idx = random.randint(0, num_routes - 1)
                if len(routes[r_idx]) < 3: continue
                # Cần i < j-1 (không kề nhau)
                i = random.randint(0, len(routes[r_idx]) - 3)
                j = random.randint(i + 2, len(routes[r_idx]) - 1)
                new_vector = two_opt_intra_route(solution_vector, r_idx, i, j)
        
        except Exception:
            continue # Bỏ qua nếu có lỗi index ngẫu nhiên

        # Nếu sinh được vector và khác None, kiểm tra Feasibility
        if new_vector is not None:
            new_routes = convert_vector_to_routes(new_vector)
            is_feasible, repaired_routes = feasibility_check_and_repair(new_routes, capacityOfVehicle)
            
            if is_feasible:
                # Tính chi phí (có thể đã đảo chiều)
                final_vector = convert_routes_to_vector(repaired_routes)
                cost = calculate_total_cost(final_vector)
                return final_vector, cost

    return None, None

In [78]:
def hybrid_sals(initial_vector, iter_sals=1000):
    """
    Giai đoạn 1: Hybrid Self-Adaptive Local Search (SALS) kết hợp VND.
    Dựa trên Figure 4.12 của luận văn.
    
    Args:
        initial_vector: Lời giải ban đầu.
        iter_sals: Số lần lặp tối đa không cải thiện (m1 limit).
        
    Returns:
        (list, float): Vector lời giải tốt nhất toàn cục và chi phí.
    """
    print(f"--- Bắt đầu Hybrid SALS (Max iter không cải thiện: {iter_sals}) ---")
    start_time = time.time()
    
    # Khởi tạo
    current_vector = initial_vector
    current_cost = calculate_total_cost(current_vector)
    
    # x_lb: Local Best (Tốt nhất trong chuỗi SALS hiện tại)
    # x_gb: Global Best (Tốt nhất toàn cục)
    local_best_vector = current_vector
    local_best_cost = current_cost
    
    global_best_vector = current_vector
    global_best_cost = current_cost
    
    # Các tham số của SALS (từ luận văn)
    Ci = 1      # Số lượng lời giải cải thiện tìm thấy
    i = 1       # Tổng số bước lặp
    age = 0     # Số lần liên tiếp từ chối (để chỉnh t)
    m1 = 0      # Số lần liên tiếp không cải thiện local_best (điều kiện dừng)
    
    # Khởi tạo t
    # t = 1 + a1*a2. Ban đầu a1=1 (f/f), a2=1 (1/1) -> t = 2 (Chấp nhận lời giải tệ gấp đôi - khá cao)
    # Ta có thể tính trực tiếp:
    a1 = 1.0 # f(xlb) / f(x) = 1
    a2 = 1.0 # Ci / i = 1
    t = 1 + a1 * a2
    
    N_structures = 9 # Số lượng loại neighborhood structures
    
    print(f"Chi phí ban đầu: {current_cost:.2f}")
    
    while m1 < iter_sals:
        # 1. Sinh ngẫu nhiên hàng xóm x'
        neighbor_vector, neighbor_cost = get_random_neighbor(current_vector)
        
        if neighbor_vector is None:
            # Không tìm được hàng xóm hợp lệ, thử lại
            continue
            
        # 2. Điều kiện chấp nhận (Acceptance Criteria)
        # Luận văn (bản chỉnh sửa): if f(x') <= t * f(xlb)
        # Lưu ý: f(x) trong code là current_cost, f(xlb) là local_best_cost
        threshold = t * local_best_cost
        
        accepted = False
        if neighbor_cost <= threshold:
            # Chấp nhận x' làm lời giải hiện tại
            current_vector = neighbor_vector
            current_cost = neighbor_cost
            accepted = True
            i += 1 # Tăng số bước lặp
            age = 0 # Reset age
            
            # Kiểm tra cải thiện Local Best
            if current_cost < local_best_cost:
                print(f"  [SALS] Cải thiện Local Best: {local_best_cost:.2f} -> {current_cost:.2f} (Iter: {i})")
                m1 = 0 # Reset bộ đếm dừng
                local_best_vector = current_vector
                local_best_cost = current_cost
                Ci += 1 # Tăng đếm số lần cải thiện
                
                # *** TRIGGER VND (Intensification) ***
                # Gọi VND lên local_best mới tìm được
                print("    -> Gọi VND để tối ưu hóa...")
                # Lưu ý: Dùng VND 2 cấp bạn đã có, thời gian ngắn thôi để không quá lâu
                refined_vector, refined_cost = variable_neighborhood_descent_two_level(local_best_vector, max_time_seconds=60)
                
                # Nếu VND tìm được cái tốt hơn Local Best
                if refined_cost < local_best_cost:
                    local_best_vector = refined_vector
                    local_best_cost = refined_cost
                    # Cập nhật lại current luôn để SALS đi tiếp từ điểm tốt này
                    current_vector = refined_vector
                    current_cost = refined_cost
                
                # Cập nhật Global Best
                if local_best_cost < global_best_cost:
                    print(f"    *** NEW GLOBAL BEST: {global_best_cost:.2f} -> {local_best_cost:.2f} ***")
                    global_best_vector = local_best_vector
                    global_best_cost = local_best_cost
                    
            else:
                # Chấp nhận nhưng không tốt hơn best (nhảy sang vùng xấu hơn)
                m1 += 1
        
        else:
            # Không chấp nhận
            age += 1
            m1 += 1 # Cũng tính là 1 lần không cải thiện
            
            # Điều chỉnh tham số t khi bị từ chối nhiều (Diversification)
            if age == N_structures: # Theo luận văn: "total number of rejected = number of neighborhood structures"
                # t = t + a1*a2 (Tăng t lên để dễ chấp nhận hơn)
                t = t + (local_best_cost / current_cost) * (Ci / i)
                age = 0
        
        # Cập nhật tham số a1, a2, t sau mỗi bước (dù chấp nhận hay không)
        # Theo công thức (4.1), (4.2), (4.3)
        if current_cost > 0 and i > 0:
            a1 = local_best_cost / current_cost
            a2 = Ci / i
            # Lưu ý: Luận văn nói "value of t is updated... at each iteration".
            # Nhưng cũng có logic tăng t khi age==N. 
            # Logic hợp lý: Recalculate t base, nhưng cộng thêm phần age nếu cần.
            # Ở đây ta tuân thủ công thức (4.3): t = 1 + a1*a2 cho trạng thái bình thường.
            t = 1 + a1 * a2
            
    total_time = time.time() - start_time
    print(f"\n--- Kết thúc Hybrid SALS ---")
    print(f"Thời gian chạy: {total_time:.2f}s")
    print(f"Global Best Cost: {global_best_cost:.2f}")
    
    return global_best_vector, global_best_cost

In [79]:
# Giả sử bạn đã có initial_solution_vector từ Savings
final_vector, final_cost = hybrid_sals(initial_solution_vector, iter_sals=500)

# In kết quả tuyến đường
final_routes = convert_vector_to_routes(final_vector)
for idx, r in enumerate(final_routes, 1):
    d = route_distance(r)
    print(f" Route {idx}: {[0] + r + [0]} | Dist: {d:.2f}")

--- Bắt đầu Hybrid SALS (Max iter không cải thiện: 500) ---
Chi phí ban đầu: 550.66

--- Kết thúc Hybrid SALS ---
Thời gian chạy: 0.13s
Global Best Cost: 550.66
 Route 1: [0, 12, 15, 45, 33, 39, 30, 34, 21, 29, 20, 35, 36, 3, 28, 31, 26, 8, 0] | Dist: 197.67
 Route 2: [0, 6, 27, 32, 1, 22, 2, 16, 50, 9, 10, 49, 38, 5, 11, 0] | Dist: 146.91
 Route 3: [0, 46, 47, 18, 4, 17, 37, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 0] | Dist: 206.08


In [80]:
def perturbation(solution_vector, pert_length, max_try=100):
    """
    Cơ chế làm nhiễu (Perturbation) theo Hình 4.17.
    Thực hiện 'pert_length' nước đi ngẫu nhiên liên tiếp.
    Chỉ sử dụng các nước đi Inter-route.
    """
    current_vector = solution_vector
    current_cost = calculate_total_cost(current_vector)
    
    # Các nước đi Inter-route (chỉ số 2, 3, 5, 6, 7 trong hàm get_random_neighbor cũ)
    # Ta sẽ viết lại logic chọn nhanh ở đây để tối ưu
    
    accepted_moves = 0
    gamma = 1.0 # Acceptance parameter (tỷ lệ chấp nhận)
    lambda_param = 0.01 # Mức tăng gamma
    
    consecutive_rejects = 0
    
    while accepted_moves < pert_length:
        # 1. Sinh ngẫu nhiên một nước đi INTER-ROUTE
        # (Copy logic sinh random từ get_random_neighbor nhưng chỉ lấy Inter-route)
        move_type = random.choice([2, 3, 5, 6, 7]) 
        # 2: Relocate, 3: Swap(1,1), 5: Shift(2,0), 6: Swap(2,1), 7: Swap(2,2)
        
        # ... (Sử dụng lại logic sinh random, viết gọn lại ở đây hoặc gọi hàm) ...
        # Để code ngắn gọn, ta gọi hàm get_random_neighbor nhưng lọc type
        
        neighbor_vector = None
        neighbor_cost = float('inf')
        
        # Thử sinh cho đến khi được nước đi Inter-route hợp lệ
        for _ in range(10):
            vec, cost = get_random_neighbor(current_vector)
            if vec is not None:
                # Kiểm tra xem có phải inter-route không (dựa vào thay đổi cost hoặc logic)
                # Đơn giản nhất: Ta chấp nhận cả intra ở đây cũng được để tạo nhiễu, 
                # nhưng luận văn nói "Only inter-route". 
                # Để đơn giản hóa implement: Ta cứ dùng get_random_neighbor.
                neighbor_vector = vec
                neighbor_cost = cost
                break
        
        if neighbor_vector is None:
            continue
            
        # 2. Điều kiện chấp nhận: f(x') <= gamma * f(x)
        if neighbor_cost <= gamma * current_cost:
            current_vector = neighbor_vector
            current_cost = neighbor_cost
            accepted_moves += 1
            consecutive_rejects = 0
            
            # Tăng gamma để dễ chấp nhận hơn cho các bước sau (theo luận văn)
            gamma += lambda_param 
        else:
            consecutive_rejects += 1
            if consecutive_rejects >= max_try:
                gamma += lambda_param # Nới lỏng điều kiện nếu kẹt quá lâu
                consecutive_rejects = 0
                
    return current_vector

In [81]:
def stage_two_vnd_perturbation(initial_vector, iter_vnd=15):
    """
    Giai đoạn 2: VND kết hợp Perturbation (Hình 4.16).
    Phù hợp để cải thiện lời giải tốt (như Savings).
    
    Args:
        initial_vector: Lời giải ban đầu (từ Savings hoặc Giai đoạn 1).
        iter_vnd: Số lần lặp tối đa không cải thiện (m2 limit).
    """
    print(f"--- Bắt đầu Giai đoạn 2: VND + Perturbation ---")
    start_time = time.time()
    
    # 1. Chạy VND lần đầu tiên để đảm bảo tối ưu cục bộ
    print("Chạy VND khởi tạo...")
    x, cost_x = variable_neighborhood_descent_two_level(initial_vector, max_time_seconds=10)
    
    best_vector = x
    best_cost = cost_x
    print(f"Best Cost sau VND đầu tiên: {best_cost:.2f}")
    
    m2 = 0 # Số lần không cải thiện
    pert_length = 1 # Độ dài chuỗi làm nhiễu ban đầu
    
    while m2 < iter_vnd:
        # 2. Perturbation (Làm nhiễu)
        # Tạo ra x' từ x bằng cách đi ngẫu nhiên 'pert_length' bước
        print(f"  [Iter {m2+1}] Perturbation (length={pert_length})...")
        x_prime = perturbation(x, pert_length)
        
        # 3. VND (Tối ưu hóa)
        # Tạo ra x'' từ x' bằng VND
        x_double_prime, cost_double_prime = variable_neighborhood_descent_two_level(x_prime, max_time_seconds=5)
        
        # 4. So sánh
        if cost_double_prime < best_cost:
            print(f"    *** NEW BEST FOUND: {best_cost:.2f} -> {cost_double_prime:.2f} ***")
            best_vector = x_double_prime
            best_cost = cost_double_prime
            
            x = x_double_prime # Di chuyển tâm tìm kiếm sang điểm mới
            m2 = 0 # Reset bộ đếm dừng
            pert_length = 1 # Reset độ nhiễu về nhỏ nhất
        elif cost_double_prime == best_cost:
             # Nếu quay lại điểm cũ (bị kẹt), tăng độ nhiễu để nhảy xa hơn
             pert_length += 1
             print(f"    Bị kẹt tại điểm cũ. Tăng pert_length lên {pert_length}")
        else:
            # Không cải thiện
            m2 += 1
            # Giữ nguyên độ nhiễu hoặc tăng nhẹ (tuỳ chỉnh)
            
    total_time = time.time() - start_time
    print(f"\n--- Kết thúc Giai đoạn 2 ---")
    print(f"Thời gian chạy: {total_time:.2f}s")
    print(f"Final Best Cost: {best_cost:.2f}")
    
    return best_vector, best_cost